# Fine-tuning Gemma 2B for Mental Health & Wellness Counseling (English)

A comprehensive notebook for fine-tuning Google's Gemma 2B model to provide:
- **Mental Health Support**: Counseling and emotional support
- **Sex Education**: Age-appropriate, evidence-based information
- **Self-Protection**: Personal safety and recognizing inappropriate behavior
- **Social Behavior**: Healthy relationships and communication skills

## Overview
This notebook provides a complete pipeline for:
1. **Data Loading**: Load curated English training dataset from JSONL files
2. **Model Fine-tuning**: Use QLoRA (4-bit quantization) for efficient training
3. **Model Testing**: Validate responses across different topics
4. **GGUF Conversion**: Deploy with Ollama for production use

## Key Features
- Memory-efficient training with 4-bit quantization
- Pre-curated dataset with comprehensive counseling examples
- Comprehensive testing and validation
- Production-ready GGUF export

**Dataset Source**: `data/english_counseling.train.jsonl` (273 examples)
**Model**: Google Gemma 2B Instruct (`google/gemma-2-2b-it`)
**Target Audience**: Young adults and teenagers seeking accessible mental health and wellness information


In [ ]:
# Mount Google Drive for persistent storage
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
    
    # Define paths for Drive
    # You can change 'nganiriza_capstone' to your preferred folder name
    DRIVE_BASE_PATH = "/content/drive/MyDrive/nganiriza_capstone"
    DATA_PATH = f"{DRIVE_BASE_PATH}/data"
    MODEL_SAVE_PATH = f"{DRIVE_BASE_PATH}/models"
    
    import os
    # Create directories if they don't exist
    os.makedirs(DATA_PATH, exist_ok=True)
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    
    print(f"Using Drive paths:\nData: {DATA_PATH}\nModels: {MODEL_SAVE_PATH}")
    
except ImportError:
    print("Google Colab not detected. Using local paths.")
    # Define local paths
    DATA_PATH = "./data"
    MODEL_SAVE_PATH = "./models"
    
    import os
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"Using local paths:\nData: {DATA_PATH}\nModels: {MODEL_SAVE_PATH}")

## Step 1: Install Required Libraries


In [ ]:
# Upgrade bitsandbytes to ensure 4-bit quantization support
%pip install -q -U bitsandbytes
%pip install -q -U transformers
%pip install -q accelerate peft datasets
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

## Step 2: Load Training Dataset from File

Loading the English counseling dataset from the JSONL file in the data folder. This dataset covers:
- **Mental Health**: Anxiety, depression, stress management, coping strategies
- **Sex Education**: Reproductive health, relationships, consent, safety
- **Self-Protection**: Personal safety, recognizing inappropriate behavior
- **Social Behavior**: Peer pressure, healthy relationships, communication


In [ ]:
import json
import os

def load_training_data_from_jsonl(file_path: str):
    """Load training data from JSONL file."""
    training_data = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line.strip())
            training_data.append({
                "instruction": data["instruction"],
                "output": data["output"]
            })
    
    return training_data

print("Loading English training dataset from file...")

# Load training data from the English JSONL file
training_file = os.path.join(DATA_PATH, "english_counseling.train.jsonl")

# Fallback to local if not found in Drive
if not os.path.exists(training_file):
    local_file = "./data/english_counseling.train.jsonl"
    if os.path.exists(local_file):
        print(f"File not found in {DATA_PATH}, using local file: {local_file}")
        training_file = local_file
    else:
        print(f"WARNING: Training file not found in {DATA_PATH} or {local_file}")

training_examples = load_training_data_from_jsonl(training_file)

print(f"\nLoaded {len(training_examples)} training examples from {training_file}")

# Display sample examples
print("\n" + "="*80)
print("SAMPLE TRAINING EXAMPLES")
print("="*80)

for i, example in enumerate(training_examples[:3], 1):
    print(f"\n--- Example {i} ---")
    print(f"Input: {example['instruction']}")
    print(f"Output: {example['output'][:200]}...")
    print()

# Store for later use
instructions = [ex['instruction'] for ex in training_examples]
responses = [ex['output'] for ex in training_examples]

print(f"Ready for training with {len(instructions)} instruction-response pairs")

## Step 3: Verify Dataset Quality

Review the loaded dataset to ensure:
- Comprehensive coverage of counseling topics
- Appropriate and helpful responses
- Natural conversation flow
- Evidence-based information


In [ ]:
# Verify dataset statistics
print("="*80)
print("DATASET VERIFICATION")
print("="*80)

print(f"\nTotal training pairs: {len(training_examples)}")
print(f"Source: {training_file}")

# Count topics if available
topics = {}
for ex in training_examples:
    # Try to extract topic from the data if it exists
    if isinstance(ex, dict) and 'topic' in ex:
        topic = ex.get('topic', 'general')
        topics[topic] = topics.get(topic, 0) + 1

if topics:
    print(f"\nTopics covered:")
    for topic, count in sorted(topics.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"   - {topic}: {count} examples")

print(f"\nSample questions:")
sample_questions = [ex['instruction'] for ex in training_examples[::max(len(training_examples)//10, 1)]]
for i, q in enumerate(sample_questions[:10], 1):
    print(f"   {i}. {q[:60]}...")

print(f"\nDataset is ready for model training")
print(f"The model will learn to:")
print(f"   - Provide mental health support and guidance")
print(f"   - Offer evidence-based sex education")
print(f"   - Share wellness and self-care strategies")
print(f"   - Support healthy relationships and communication")


## Step 4: Format Data for Gemma Model

Gemma uses a specific chat template format with `<start_of_turn>` and `<end_of_turn>` tokens.

In [ ]:
def format_for_training(instruction: str, response: str) -> str:
    """Format data in Gemma's chat template format."""
    return f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n{response}<end_of_turn>"

# Format all training examples
print("Formatting data for Gemma model...")
formatted_data = [
    format_for_training(ex['instruction'], ex['output']) 
    for ex in training_examples
]

print(f"Formatted {len(formatted_data)} examples")
print(f"\nSample formatted conversation:\n")
print(formatted_data[0][:300] + "...")


## Step 5: Create Hugging Face Dataset

In [ ]:
from datasets import Dataset

# Create Hugging Face dataset
dataset = Dataset.from_dict({"text": formatted_data})

print(f"Created dataset with {len(dataset)} examples")
print(f"Dataset structure: {dataset}")
print(f"\nReady for tokenization and training")


## Step 6: Load Gemma Model with 4-bit Quantization

Using QLoRA (Quantized Low-Rank Adaptation) for memory-efficient fine-tuning.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
import torch

# Configuration
MODEL_NAME = "google/gemma-2-2b-it"  # Gemma 2B - efficient and capable

print("Logging in to Hugging Face...")
login()

# 4-bit quantization config for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print(f"\nLoading tokenizer from {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Loading model with 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

print(f"\nModel loaded successfully!")
print(f"   Model: {MODEL_NAME}")
print(f"   Dtype: {model.dtype}")
print(f"   Device: {model.device}")
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
import torch

# Configuration
MODEL_NAME = "google/gemma-2-2b-it"  # Gemma 2B - efficient and capable

print("Logging in to Hugging Face...")
login()

# 4-bit quantization config for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print(f"\nLoading tokenizer from {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Loading model with 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

print(f"\nModel loaded successfully!")
print(f"   Model: {MODEL_NAME}")
print(f"   Dtype: {model.dtype}")
print(f"   Device: {model.device}")



## Step 7: Configure PEFT (Parameter-Efficient Fine-Tuning) with LoRA

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,  # LoRA alpha scaling
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()


## Step 8: Tokenize Dataset

In [ ]:
def tokenize_data(examples):
    """Tokenize examples for causal language modeling."""
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_data,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Tokenizing"
)

print(f"Tokenization complete!")
print(f"   Examples: {len(tokenized_dataset)}")
print(f"   Features: {tokenized_dataset.features}")
print(f"   Sample shape: {len(tokenized_dataset[0]['input_ids'])} tokens")


## Step 9: Configure Training Arguments

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

# Optimized training configuration
training_args = TrainingArguments(
    output_dir=os.path.join(MODEL_SAVE_PATH, "counseling-model-checkpoints"),
    
    # Training schedule
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    
    # Optimization
    learning_rate=2e-4,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    max_grad_norm=0.3,
    
    # Precision
    bf16=True,  # bfloat16 for Gemma
    fp16=False,
    
    # Logging & Saving
    logging_steps=5,
    save_steps=25,
    save_total_limit=2,
    
    # Reporting
    report_to="none",  # Disable external reporting for simplicity
    logging_first_step=True,
)

# Data collator for causal LM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM, not masked
)

print("Training configuration ready!")
print(f"\nTraining Settings:")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"   Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Total training steps: ~{len(tokenized_dataset) * training_args.num_train_epochs // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")

## Step 10: Initialize Trainer and Start Fine-tuning

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("Starting model fine-tuning...")

# Train the model
trainer.train()

print("Fine-tuning completed successfully!")
print(f"Training metrics saved to: {training_args.output_dir}")


## Step 11: Save the Fine-tuned Model

In [ ]:
# Save the fine-tuned model
OUTPUT_DIR = os.path.join(MODEL_SAVE_PATH, "counseling-model-final")

print(f"Saving fine-tuned model to {OUTPUT_DIR}...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Model saved successfully!")
print(f"Location: {OUTPUT_DIR}")
print(f"\nNext: Test the model's performance")

## Step 12: Test the Fine-tuned Model

In [ ]:
from peft import PeftModel

print("Loading fine-tuned model for testing...")

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

# Load fine-tuned weights
fine_tuned_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
fine_tuned_model.eval()

print("Fine-tuned model loaded and ready for inference!")


## Step 13: Generate Responses


In [ ]:
def generate_response(prompt: str, max_new_tokens: int = 256, temperature: float = 0.7) -> str:
    """Generate response from the fine-tuned model."""
    # Format in Gemma chat template
    formatted_prompt = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    
    # Tokenize
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(fine_tuned_model.device)
    
    # Generate
    with torch.no_grad():
        outputs = fine_tuned_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # Decode and extract response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    if "<start_of_turn>model\n" in full_response:
        response = full_response.split("<start_of_turn>model\n")[-1]
        response = response.split("<end_of_turn>")[0].strip()
    else:
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response

# Comprehensive test prompts
test_prompts = [
    # Mental Health
    "I've been feeling really anxious",
    "I'm stressed about work",
    "I feel sad all the time",
    
    # Sex Education
    "What is consent?",
    "How do I know if I'm ready for sex?",
    
    # Self-Protection
    "What should I do if someone touches me inappropriately?",
    "How do I say no when someone makes me uncomfortable?",
    
    # Social Behavior
    "I'm being bullied at school. What should I do?",
    "How can I manage stress?",
    "What are some self-care activities?",
]

print("="*80)
print("TESTING FINE-TUNED MODEL")
print("="*80)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n{'='*80}")
    print(f"Test {i}/{len(test_prompts)}")
    print(f"{'='*80}")
    print(f"User: {prompt}")
    print(f"\nAssistant:")
    
    response = generate_response(prompt, max_new_tokens=200)
    print(response)
    print()

print("="*80)
print("Testing complete!")


In [ ]:
print("="*80)
print("PART 1 COMPLETE: MODEL FINE-TUNING")
print("="*80)

print(f"\nSuccessfully completed:")
print(f"   1. Loaded {len(training_examples)} training examples from dataset file")
print(f"   2. Fine-tuned Gemma 2B model on counseling data")
print(f"   3. Tested model with diverse prompts")
print(f"   4. Saved model to: {OUTPUT_DIR}")

print(f"\nNext Steps:")
print(f"   - Part 2: Convert model to GGUF format")
print(f"   - Deploy with Ollama for production use")
print(f"\n{'='*80}\n")


---

# Part 2: Convert to GGUF Format for Ollama

This section merges the LoRA adapters with the base model and converts to GGUF format for Ollama deployment.


## Step 14: Install System Dependencies


In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq build-essential git cmake
!pip install -q gguf sentencepiece protobuf

print("Dependencies installed")


## Step 15: Setup llama.cpp for GGUF Conversion


In [ ]:
import os

# Clone llama.cpp if needed
if not os.path.exists("llama.cpp"):
    !git clone https://github.com/ggerganov/llama.cpp.git

# Install requirements and build
!pip install -q -r llama.cpp/requirements.txt
!make -C llama.cpp

print("llama.cpp ready")


## Step 16: Merge LoRA Adapters with Base Model


In [ ]:
from peft import PeftModel
import torch
import os

# Configuration
MERGED_MODEL_PATH = os.path.join(MODEL_SAVE_PATH, "counseling-model-merged")

# Load base model
merge_base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# Load and merge LoRA adapters
peft_model = PeftModel.from_pretrained(merge_base, OUTPUT_DIR)
merged_model = peft_model.merge_and_unload()

# Save merged model
os.makedirs(MERGED_MODEL_PATH, exist_ok=True)
merged_model.save_pretrained(MERGED_MODEL_PATH)
tokenizer.save_pretrained(MERGED_MODEL_PATH)

print(f"Model merged and saved to: {MERGED_MODEL_PATH}")

# Cleanup memory
del merge_base, peft_model, merged_model
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## Step 17: Convert to GGUF Format


In [ ]:
import subprocess
import sys
import os

# Configuration
GGUF_OUTPUT = os.path.join(MODEL_SAVE_PATH, "counseling-model.gguf")
QUANTIZATION = "Q4_K_M"

# Find conversion script
convert_script = "./llama.cpp/convert_hf_to_gguf.py"
if not os.path.exists(convert_script):
    convert_script = "./llama.cpp/convert.py"

# Build conversion command
cmd = [
    sys.executable, convert_script,
    MERGED_MODEL_PATH,
    "--outtype", QUANTIZATION,
    "--outfile", GGUF_OUTPUT
]

# Run conversion
print(f"Converting to GGUF ({QUANTIZATION})...")
result = subprocess.run(cmd, check=True, capture_output=True, text=True)

# Check output
if os.path.exists(GGUF_OUTPUT):
    size_gb = os.path.getsize(GGUF_OUTPUT) / (1024**3)
    print(f"GGUF file created: {size_gb:.2f} GB")
else:
    print("Conversion failed - file not created")

## Step 18: Create Modelfile for Ollama


In [ ]:
# Create Modelfile for Ollama
modelfile_content = """FROM ./counseling-model.gguf

PARAMETER temperature 0.7
PARAMETER top_p 0.95
PARAMETER top_k 40
PARAMETER num_ctx 2048
PARAMETER repeat_penalty 1.1

SYSTEM \"\"\"You are a compassionate mental health and wellness counseling assistant. You provide:

- Supportive, empathetic mental health guidance
- Evidence-based sex education information
- Wellness and self-care strategies
- Friendly conversation and emotional support

Guidelines:
- Be warm, approachable, and non-judgmental
- Provide accurate, evidence-based information
- Encourage professional help for serious concerns
- Maintain appropriate boundaries
- Respect user privacy and autonomy

Important: You provide educational support, not professional therapy. Always encourage users to seek qualified professionals for serious mental health concerns.\"\"\"

TEMPLATE \"\"\"{{- if .System }}<start_of_turn>system
{{ .System }}<end_of_turn>
{{ end }}<start_of_turn>user
{{ .Prompt }}<end_of_turn>
<start_of_turn>model
{{ .Response }}\"\"\"

LICENSE "Apache 2.0"
"""

with open("./Modelfile", 'w') as f:
    f.write(modelfile_content)

print("Modelfile created")


## Step 19: Create Usage Instructions


In [ ]:
# Create usage guide
usage_content = """# Counseling Assistant - Usage Guide

## Quick Start

1. Install Ollama: https://ollama.ai
2. Create model: `ollama create counseling-assistant -f Modelfile`
3. Run model: `ollama run counseling-assistant`

## Example Usage

```bash
ollama run counseling-assistant "I'm feeling stressed"
ollama run counseling-assistant "What is consent?"
ollama run counseling-assistant "How can I manage anxiety?"
```

## Model Information

- Base: Google Gemma 2B Instruct
- Training: 273 English counseling examples
- Format: GGUF Q4_K_M (~1.5-2GB)
- Topics: Mental health, sex education, safety, relationships

## Important Disclaimer

This AI provides educational support only - NOT professional therapy.

**Crisis Resources:**
- US: 988 (Suicide & Crisis Lifeline)
- Text: 741741 (Crisis Text Line)
- Emergency: 911/999/112

## Troubleshooting

```bash
ollama list  # Check models
ollama rm counseling-assistant  # Remove model
ollama create counseling-assistant -f Modelfile  # Recreate
```
"""

with open("./usage_instructions.md", 'w') as f:
    f.write(usage_content)

print("Usage guide created")


## Step 20: Deployment Summary


In [ ]:
import os

print("\n" + "="*80)
print("DEPLOYMENT READY")
print("="*80)

# Check files
files = [
    ("counseling-model.gguf", "GGUF model"),
    ("Modelfile", "Ollama config"),
    ("usage_instructions.md", "Usage guide"),
]

print("\nFiles:")
for filename, desc in files:
    if os.path.exists(filename):
        size = os.path.getsize(filename) / (1024**2)
        print(f"  [+] {filename} ({size:.1f} MB)")
    else:
        print(f"  [-] {filename} - NOT FOUND")

print("\nDeploy with Ollama:")
print("  ollama create counseling-assistant -f Modelfile")
print("  ollama run counseling-assistant")

print("\nModel: Gemma 2B | Training: 273 examples | Format: GGUF Q4_K_M")
print("="*80 + "\n")


## Notes & Best Practices

### Dataset
- **Source**: Pre-curated JSONL file (`data/english_counseling.train.jsonl`)
- **Size**: 273 comprehensive training examples
- **Coverage**: Mental health, sex education, social behavior, self-protection
- **Language**: English with clear, accessible language for broad reach
- **Evidence-Based**: All health information follows current best practices
- **Expandable**: Easy to add more examples to the JSONL file

### Model
- **Base Model**: Google Gemma 2B Instruct (`google/gemma-2-2b-it`)
- **Size**: 2 billion parameters - efficient for consumer hardware
- **Architecture**: Instruction-tuned for chat/counseling tasks

### Training Efficiency
- **QLoRA (4-bit)**: Reduces memory usage by ~75% vs full precision
- **Small Batch Size**: Works on consumer GPUs (8GB+ VRAM)
- **Gradient Accumulation**: Simulates larger batches efficiently
- **Fast Training**: ~10-30 minutes depending on hardware

### Memory Management
If you encounter OOM (Out of Memory) errors:
- Reduce `per_device_train_batch_size` to 2 or 1
- Increase `gradient_accumulation_steps` to maintain effective batch size
- Use smaller model (Gemma 2B is already optimized)
- Close other GPU applications

### Model Performance
- **Mental Health**: Provides supportive, evidence-based guidance
- **Sex Education**: Offers accurate, judgment-free information
- **Social Skills**: Supports healthy relationships and communication
- **Safety**: Recognizes inappropriate behavior and encourages seeking help
- **Boundaries**: Appropriately encourages professional help when needed

### GGUF Conversion
- **Q4_K_M Quantization**: Best balance of size and quality
- **Alternative Options**: 
  - Q2_K: Smaller file, lower quality
  - Q5_K_M: Larger file, better quality
  - Q8_0: Largest, highest quality
- **File Size**: ~1.5-2GB for Q4_K_M (2B model)

### Deployment
- **Ollama**: Easiest option for local deployment
- **Production**: Can integrate into web apps, chatbots, mobile apps
- **Privacy**: Runs entirely locally, no data sent externally
- **Disclaimers**: Always include appropriate mental health disclaimers

### Ethical Considerations
1. **Not a Replacement**: Clearly state this is not professional therapy
2. **Crisis Resources**: Include crisis hotline information
3. **Evidence-Based**: All advice follows current best practices
4. **Non-Judgmental**: Maintains supportive, accepting tone
5. **Boundaries**: Knows limitations and encourages professional help

### Future Improvements
- Add validation dataset (`english_counseling.validation.jsonl`)
- Include multilingual support (Kinyarwanda dataset available)
- Fine-tune for specific age groups or demographics
- Integrate with actual mental health resources
- Add crisis detection and immediate resource provision
